# Marine diffusion

```{admonition} Diffusion approach


In the marine realm, a diffusion model is used for sediment-transport by rivers. When the dual lithology is activated, it accounts for distinct transport coefficients for the two different grain sizes.

During a single time step, marine sediment transport is performed until all available sediment transported by rivers to the ocean have been diffused and the accumulations on these specific nodes remains below water depth or below a prescribed slope computed based on local water depth and distance to the nearest coastline.

The coarser sediments are deposited first, followed by the finer ones. It allows for finer sediments to be deposited further and reproduce the standard behaviour observed in stratigraphic architectures. 
```

```{caution}
In this notebook, we test a simple 1D version of the algorithm implemented in **[gospl](https://gospl.readthedocs.io/en/latest/index.html#module-gospl)**. There are several limitations in the approach presented below.
```

In [ ]:
import numpy as np
import pandas as pd
from scipy import linalg

import script.toolGrid as tools

import matplotlib
import matplotlib.pyplot as plt

from scipy.spatial import cKDTree

label_size = 7
matplotlib.rcParams['xtick.labelsize'] = label_size
matplotlib.rcParams['ytick.labelsize'] = label_size
matplotlib.rc('font', size=6)

%config InlineBackend.figure_format = 'svg'
%matplotlib inline

## Definition of a cross-section

The 1D diffusion model requires a 2D coordinates vector containing the point position along the transect (equally spaced x-values) and the elevations of these points. 

Defining a generic cross-section is simple, here we will show how to extract one from  eTOPO.

First we define a bounding box of interest based on lower left and upper right longitude and latitudes coordinates:

In [ ]:
grid = tools.toolGrid(llcrnrlon = -100, llcrnrlat = 19, 
                      urcrnrlon = -80,  urcrnrlat = 33.)

grid.getSubset(tfile = 'etopo1', offset = 0.1, smooth = False)

grid.mapUTM(contour=250, fsize=(10,8))

### Extract section from eTOPO

We then define the position of the start and end point of the cross-section. 

```{important} 
For the 1D model to work it is required to have the first coordinate of the section to be inland and the second to be under water.
```

Then, we extract the section distance and elevation.

In [ ]:
# First point position (inland)
pt1 = [-92,30]
# Second point position (marine)
pt2 = [-90,23]

# Resolution to use in metres
res = 10000.

# Extract the section 2D coordinates
XY = np.column_stack((grid.x.flatten(), grid.y.flatten()))
Z = grid.topo.flatten()
lonlat = np.column_stack((grid.lon.flatten(), grid.lat.flatten()))
tree = cKDTree(lonlat)

dist1, id1 = tree.query(pt1, k=1)
dist2, id2 = tree.query(pt2, k=1)

x1, y1 = XY[id1][0],XY[id1][1]
x2, y2 = XY[id2][0],XY[id2][1]

dist = np.sqrt(( x2 - x1 )**2 + ( y2 - y1 )**2)
nbpts = int(dist/res)+1

if x2 != x1:
    a = (y2-y1)/(x2-x1)
    b = y1 - a * x1
    xsec = np.linspace(x1, x2, nbpts)
    ysec = a * xsec + b
else:
    xsec = np.zeros(nbpts)+x1
    ysec = np.linspace(y1, y2, nbpts)
    
xysec = np.column_stack((xsec.flatten(), ysec.flatten()))
sec = np.sqrt(( xsec - x1 )**2 + ( ysec - y1 )**2)
dx = np.round(sec[1]-sec[0],1)
secdist = np.arange(len(sec))*dx

tree = cKDTree(XY)
distances, indices = tree.query(xysec, k=3)
offIDs = np.where(distances[:,0] > 0)[0]
onIDs = np.where(distances[:,0] == 0)[0]

secZ = np.zeros(len(xysec))
if len(offIDs) > 0:
    secZ[offIDs] = np.average(Z[indices][offIDs,:],weights=(1./distances[offIDs,:]), axis=1)

if len(onIDs) > 0:
    secZ[onIDs] = Z[indices[onIDs,0]]
    
shore = np.where(secZ<0)[0][0]

### Visualise the cross-section

We can plot our cross-section:

In [ ]:
bottom = np.zeros(len(secdist))
plt.figure(figsize=(8,3))
ax = plt.gca()

# Ocean
ax.fill_between(secdist, bottom-10000., bottom, color='lightblue')

# Initial basement
ax.fill_between(secdist, bottom-10000., secZ, color='lightgray')
plt.plot(secdist, secZ, lw=1, c='k')

# Shore line
plt.plot(secdist[shore],0, 'o', color='r', markersize=4, 
         markeredgecolor='r', markeredgewidth=1)

plt.xlim(0,secdist.max())
plt.ylim(-5000, 300)
plt.tight_layout()
plt.show()

### Save it to file

In [ ]:
df = pd.DataFrame({'X':secdist.flatten(),'Z':secZ.flatten()})
df.to_csv('gom_section.csv',columns=['X', 'Z'], sep=' ', index=False ,header=0)

## Diffusion algorithm functions

An implicit finite volume approach is implemented and follow solution proposed for the heat equation.


```{admonition} Some usefull links on algorithms
:class: dropdown

- [Numerical Modeling of Earth Systems](http://www-udc.ig.utexas.edu/external/becker/Geodynamics557.pdf) section 4.6.1
- [Xiaoting Gu PhD](https://etd.ohiolink.edu/apexprod/rws_etd/send_file/send?accession=case1295638232&disposition=attachment)
- [Solving nonlinear ODE and PDE problems](https://hplgit.github.io/num-methods-for-PDEs/doc/pub/nonlin/pdf/nonlin-4print.pdf)
- [Steady heat conduction 2D](https://skill-lync.com/projects/Solving-the-2-D-steady-and-unsteady-heat-conduction-equation-using-finite-difference-explicit-and-implicit-iterative-solvers-in-MATLAB-37587)

```

In [ ]:
# Compute diffusion coefficients 
def compute_Ki(D, h, zb, dx, sl):
    
    dh = h-zb
    Khalf = np.zeros(len(h))
    Khalf_ = np.zeros(len(h))
    
    for k in range(1,len(h)-1):
        tmpz = 0.5*(zb[k+1]+zb[k])
        if tmpz < sl:
            tmph = 0.5*(dh[k+1]+dh[k])
            if tmph > 0.1:
                Khalf[k] = D*(1.-np.exp(-tmph/100.))
                if h[k] < h[k+1] and dh[k+1] == 0.:
                    Khalf[k] = 0.
                if h[k] > h[k+1] and dh[k] == 0.:
                    Khalf[k] = 0.
                    
        tmpz = 0.5*(zb[k-1]+zb[k])
        if tmpz < sl:
            tmph = 0.5*(dh[k-1]+dh[k])
            if tmph > 0.1:
                Khalf_[k] = D*(1.-np.exp(-tmph/100.)) 
                if h[k] < h[k-1] and dh[k-1] == 0.:
                    Khalf_[k] = 0.
                if h[k] > h[k-1] and dh[k] == 0.:
                    Khalf_[k] = 0.
    
    Khalf[0] = 0.
    Khalf[-1] = 0.
    Khalf_[0] = 0.
    Khalf_[-1] = 0.
    
    return Khalf_, Khalf

# Define PDE right and left handside
def get_System(D, h, dx, dt, zb, sl):
    
    Khalf_, Khalf = compute_Ki(D, h, zb, dx, sl)
    matD = np.zeros((len(h),len(h)))
    
    for k in range(1,len(h)-1):
        matD[k,k] = 1.+dt*(Khalf[k]+Khalf_[k])/(dx**2)
        matD[k,k-1] = -dt*(Khalf_[k])/(dx**2)
        matD[k,k+1] = -dt*(Khalf[k])/(dx**2)
    
    matD[0,0] = 1.
    matD[-1,-1] = 1.
    
    RHS = np.zeros(len(h))
    for k in range(0,len(h)):
        RHS[k] = h[k]
    
    return matD, RHS

# Solve the diffusion equation for one time step
def run_onestep(D, dx, dt, base, hsed, sl, perc, slp):
    
    solh = hsed + base
    sumsed = np.sum(hsed)
    base0 = base.copy()
    nbase = base.copy()
    
    # Clinoform slope
    maxh = base.copy()
    id = np.where(base>sl)[0]
    slps = np.arange(0,len(maxh)-id[-1])*slp*dx 
    maxh[id[-1]:] = slps
    maxh[id[-1]:] += base[id[-1]]
    maxh = np.maximum(maxh,base)
    
    excess = sumsed
    while excess>0.: 

        err = 1.e8
        while err>1.e-4:
            matD, RHS = get_System(D, solh, dx, dt, nbase, sl)
            tmp =  linalg.solve(matD, RHS)
            err = 1.0-np.max(tmp-base0)/np.max(solh-base0)
            solh = tmp.copy()
    
        # Get excess sediment
        excessID = np.where(solh>maxh)[0]
        excess = np.sum(solh[excessID]-maxh[excessID])
        solh[excessID] = maxh[excessID]
        
        if excess>perc*sumsed:
            dh = maxh-solh
            id = np.where(dh>0)[0]
            if len(id)>0:
                id = id[0]
                print(id,excess)
                nbase[excessID] = solh[excessID]
                solh[id] += excess
            else:
                excess = 0                
        else:
            excess = 0.
            step = 0
            while step<50:
                err = 1.e8
                while err>1.e-3:
                    matD, RHS = get_System(D, solh, dx, dt, base0, sl)
                    tmp =  linalg.solve(matD, RHS)
                    err = 1.0-np.max(tmp-base0)/np.max(solh-base0)
                    #err = np.max(np.abs(tmp-solh))/np.max(np.abs(tmp))
                    solh = tmp.copy()
                step += 1    
        
    return solh

# Plot diffused sediment distribution over the initial cross-section
def plot_layers(x, base, id, sl, xmin, xmax, ymin, ymax):
    
    bottom = np.zeros(len(x))+sl
    plt.figure(figsize=(8,3))
    ax = plt.gca()

    # Ocean
    ax.fill_between(x, bottom-10000., bottom, color='lightblue')
    
    # Shoreline
    plt.plot(x[id],sl, 'o', color='r', markersize=4, 
             markeredgecolor='r', markeredgewidth=1)
    
    # Initial basement
    ax.fill_between(x, bottom-10000., base[0], color='lightgray')
    
    # Deposited layers
    ax.fill_between(x, base[0], base[-1], color='gold')

    for k in range(len(base)):
        plt.plot(x, base[k], lw=1, c='k')

    plt.xlim(xmin, xmax)
    plt.ylim(ymin, ymax)
    plt.tight_layout()
    plt.show()
    
    return

# Main entry point for running sediment diffusion
def diffuseMarine(nbstep, xsec, myzb, sedh, D, dt, sealvl, perc, slp, xmin, xmax, ymin, ymax):
    
    base = []
    base.append(myzb)

    sl = sealvl
    id = np.where(base[0]<sealvl)[0] 
    dx = xsec[1]-xsec[0]

    for k in range(nbstep):

        print(k, 'elev injection point', id[0], base[-1][id[0]]) #, sl)
        hsed = np.zeros(len(base[0]))
        hsed[id[0]] += sedh

        sol = run_onestep(D, dx, dt, base[-1], hsed, sl, perc, slp)
        base.append(sol)

        ide = np.where(base[k+1]<base[k])[0]
        if len(ide)>0:
            base[k+1][ide] = base[k][ide]

        ide = np.where(base[k+1]>base[k])[0]
        if len(ide)>0:
            sumh = np.sum(base[k+1][ide]-base[k][ide])
            frac = sumh/sedh
            if frac>1.0-perc:
                frac = sumh/((1.0-perc)*sedh)
#                 print('frac',frac)
                dh = (base[k+1][ide]-base[k][ide])/frac
                base[k+1][ide] = base[k][ide]+dh
#                 print('sum',np.sum(base[k+1][ide]-base[k][ide]),(1.0-perc)*sedh)

        id2 = np.where(base[k+1]<=sl)[0] 
        if len(id2) == 0:
            print('excess deposit',np.sum(base[k+1][id2]))
            break

        id = id2
    
    plot_layers(xsec, base, id[0], sl, xmin, xmax, ymin, ymax)
        
    return base

## Test diffusion approach

### Reading the cross-section

We first read the cross-section coordinates from our `CSV` file:

In [ ]:
sec_df = pd.read_csv('gom_section.csv', sep=r'\s+', engine='c', 
                     header=None, na_filter=False, 
                     dtype=float, low_memory=False)

myx = sec_df.values[:,0]
myzb = sec_df.values[:,1]
dx = myx[1] - myx[0]

### Set diffusion model parameters

In [ ]:
# Number of time steps to run
nbstep = 5
# Individual time step duration (years)
dt = 1000.
# Sea-level position (m)
sealvl = -10.
# Thickness of sediment carried by rivers to the mouth (m3/m2)
sedh = 5000.

# Diffusion coefficient
D = 3000.

# Maximum delta slope
slp = -1.e-4

# Percentage of transported sediment that remains in suspension
perc = 0.1

### Run the 1D model

In [ ]:
# Set plot X and Y axis bounds
xmin = myx.min()
xmax = myx.max()
ymin = -4500 
ymax = 300 

newh = diffuseMarine(nbstep, myx, myzb, sedh, D, dt, sealvl, perc, slp, xmin, xmax, ymin, ymax)

Changing the diffusion coefficient:

In [ ]:
D = 100.

newh = diffuseMarine(nbstep, myx, myzb, sedh, D, dt, sealvl, perc, slp, xmin, xmax, ymin, ymax)